# What I did

At its core, this module allows the user to import images, modify them in various ways, while viewing their progress, and save the result to a new image file. But how did I do that?

## Image Storage



One of the first questions I considered was how to store the images within the glitcher object. There was no singular format that would work for everything I wanted to be able to do to the images. 
- For some methods, such as shuffling the bytes of the image file, the image needed to be encoded in a format such as JPEG. 
- For other methods, such as changing the contrast, I wanted to take advantage of the Python Image Library (PIL), which required the image representation to be stored as a PIL Image object. 
- For the bulk of my methods, I wanted to be able to interact with the pixels of the image as plainly as possible, which meant storing it as a three dimensional Numpy array: two dimensions for the width and height, and the third dimension for the RGB values of each pixel. Since I was writing this in Python, it was important to use a package such as Numpy, since for most actions, manipulating arrays in Numpy is significantly faster than manipulating them item by item as Python lists.

To handle these various storage methods, and the conversion between them, I created a new class, called ImageStorage, that would handle all the conversion. When the Glitcher needed the image to be in that format, it would call call the ImageStorage method to fetch the image in that format. If the image was already in that format, the ImageStorage object would just return that image. Otherwise, it would conver to the needed format.

But between Numpy arrays, PIL Images, and bytes for JPEG, PNG, and BMP image encodings, we now have $5$ formats, and we need to be able to convert from any one of these methods to any other one. If we need a separate function to convert from each method to each other method, we're looking at an unwieldy amount of code. Additionally, if we add a new format— say, we want to be able to work with GIF encoded images— we don't want to have to write separate methods to convert from that format to every other format.



<img src="img/conversion_problem.jpg" alt="All formats, with double arrows connecting each to each other. Very cluttered." style="width: 500px;"/>


To solve this problem, we use PIL Image objects as a kind of middle ground. There are already built in methods for converting between PIL Images and numpy arrays. PIL is built to be able to read and write image files, so we can modify that easily to allow it to read and write to byte objects instead. Now, when converting between two representations, we just convert to a PIL image, and then to the other format.

<img src="img/conversion_solution.jpg" alt="The formats go through PIL, now." style="width: 500px;"/>

## Checkpoints

I wanted playing with Glitcher to feel freeing. Glitches are generally associated with things breaking, and so can be tense experiences. With this project, however, I wanted to create code that allowed people to experiment without  anxiety.

To make that happen, it was important to have a way to undo actions. I implemented this as a list of copied Glitcher objects— essentially just a shell around the corresponding ImageStorage representaiton. To set a checkpoint, the code just makes a new copy and pushes it on the end of the list. To undo to the most recent checkpoint, it pops the last element off the list, and sets that objects image as the current image.

To make it easier to go back a large number of checkpoints, I give the user an option of giving checkpoints a name, which is stored alongside the copy in the list. When the user reverts to a named checkpoint, the code searches through the list for that name, discards all checkpoints after the name, and sets that checkpoint image as the current image.

There is also a method for making a copy of a Glitcher object. As it stands, the copy does not have the checkpoints of the original, although I may implement that in the future.

## Cellular Automata

Cellular automata can be seen as a kind of game, with pixels as players. Each pixel can be in one of two stats: either on or off. Every round of the game, the pixels check their state, and the states of their neighbors. They look that information up in a specific table (this table is the "rule" of that particular game), and turn on or off accordingly.

There are one- and two-dimensional versions of this game. A famous two-dimensional rule is known as Conway's Game of Life. The fascinating thing about this game is that despite it's simplicity, it is possible, though very difficult to build complex structures. For instance, in 2018, a group successfully built a working game of Tetris within Conway's Game of Life itself. https://codegolf.stackexchange.com/questions/11880/build-a-working-game-of-tetris-in-conways-game-of-life 

The one-dimensional version of the game is usually represented on a two-dimensional grid, with each row corresponding to one generation, with earlier generations at the top. As an example, here is the rulebook for a particular rule, called Rule 154, that I found to work well with my images:

<img src="img/rule154.jpg" alt="The formats go through PIL, now." style="width: 500px;"/>

The example below shows Rule 154 in action. The dark brown pixels correspond to the black "on" pixels in the rulebook above, and the light blue pixels correspond to the white "off" pixels. If you look at any three consecutive pixels, you should be able to see that one of the rows in the rulebook correspond to those pixels, and that the pixel below them is the color indicated by the rulebook.

<img src="img/rule154detail.png" alt="Close up illustrating rule 154 and emerging patterns." style="width: 700px;"/>

Cellular automata have fascinated me for a long time, because of the amazing complexity that's able to ememrge from such simple rules. True, the cellular automata algorithms that I implemented in Glitcher don't resemble any sort of computer glitch I've ever heard of. That said, these algorithms embody the spirit of glitch art. Glitch art is about exploration, and about producing unexpected results from computer algorithms. From the simple rules of  As much as I've studied cellular automata, I always discover new things. The collaborative nature of cellular automata— with each "player" following simple rules, but producing something larger and unexpected— also speaks to my philosophies about the human-computer interactions in glitch art.

Since this is the techical chapter, I suppose I need to give a bit of background into how I created these images. Essentially, I layered each generation over a new line of pixels in the image. When a pixel was "on," I inverted the colors of the image— so that white would become black, magenta would become green, and so forth. You can see that in the example above: the inversion of light blue/gray (the initial color) is dark orange/brown. However, as with any game, it isn't enough to have rules for how to play: you also need rules for how to set up the game to get ready to play. I decided to start with all pixels in the off state (so there are no changes to the color), but if a pixel is light enough, it flips to the on state (where the color will be inverted), at which point it will influence future pixels as the rules dictate.

In the full image below, we can see how Rule 154 affects this image of a seagull on a rock. (Note that I rotated this image 180 degrees, because I liked how that looked. As a result, the first round of the game is at the bottom, and the rounds move up instead of down.)

<img src="img/automata_seagull.png" alt="Full image, glitched with rule 154." style="width: 100%;"/>

## Playing with Image Encoding

This is the part where it gets a little rough. A lot of great glitch art is created by opening images in a text editor, and manipulating them as text. Image encodings are complicated, and I found editing them to be one of the most unpredictable parts of this project. 

To compound that, it is very easy to change the data in such a way that programs will be unable to display them. I found that some images would not display in the Preview app (the default image viewer on Apple computers), but would display when opened in Google Chrome. This was especially the case with .bmp images, since that format is primarily supported on Microsoft Windows computers. Some changes to the file rendered them unreadable in any program.